In [8]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma


loader = PyMuPDFLoader("../data/annurev-orgpsych/Brass_2022_Annurev-Orgpsych_New Developments in Social Network Analysis.pdf")
text = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 50)
text_splits = splitter.split_documents(text)

vectorstore = Chroma.from_documents(documents=text_splits, embedding=OpenAIEmbeddings())

In [19]:
text[0].page_content

'Annual Review of Organizational Psychology and\nOrganizational Behavior\nNew Developments in Social\nNetwork Analysis\nDaniel J. Brass\nLINKS Center for Social Network Analysis, Department of Management, Gatton College\nof Business and Economics, University of Kentucky, Lexington, Kentucky, USA;\nemail: dbrass@uky.edu\nAnnu. Rev. Organ. Psychol. Organ. Behav. 2022.\n9:225–46\nFirst published as a Review in Advance on\nOctober 26, 2021\nThe Annual Review of Organizational Psychology and\nOrganizational Behavior is online at\norgpsych.annualreviews.org\nhttps://doi.org/10.1146/annurev-orgpsych-012420-\n090628\nCopyright © 2022 by Annual Reviews.\nAll rights reserved\nKeywords\nsocial networks, structural holes, brokerage, negative ties, multiplex ties,\ndormant ties\nAbstract\nThis review of social network analysis focuses on identifying recent trends\nin interpersonal social networks research in organizations, and generating\nnew research directions, with an emphasis on conceptual foun

In [22]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible. 
Always say "thanks for asking!" at the end of the answer. 

{context}

Question: {question}
Helpful Answer:"""

question = "Who authored the paper?"


QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

llm = ChatOpenAI(model_name="gpt-4-0613", temperature=0)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)


result = qa_chain({"query": question, "context":"sfkdsjf;lakjfd aksd fl;askdfj"})
result["result"]

InvalidRequestError: The model `gpt-4-0613` does not exist or you do not have access to it. Learn more: https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4.